<a href="https://colab.research.google.com/github/TheBiotechGuy/monster_data_science_assignment/blob/master/Assignment_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing required libraries
import json
import re
import pandas as pd
import math
from bson.json_util import dumps
from bson import json_util, ObjectId

In [0]:
def read_json(file_path):
  """
  This function reads the json file using json and bson packages
  
  Parameters
  ----------
  file_path: Path of input json file
  
  Returns
  -------
  data: Iterable object over all the json data
  
  """
  with open(file_path, encoding='utf-8', errors='ignore') as data_file:
    data = json.loads(json_util.dumps(data_file))
  return data

In [0]:
data = read_json('searchjson.json')

In [0]:

def get_all_key_combinations_all_rows(data):
  
  """
  After reading the json file using read_json function, you might want to look at what all keys each entry has.
  
  Are these keys same or different in all the rows ?
  
  This function will return all unique key combinations across all entries in json file
  
  Parameters
  ----------
  
  data: Iterable object after reading the json file
  
  Returns
  -------
  List of all the the unique key combinations across all entries in json file
  
  """
  updated_data = []
  first_entry_data_keys = set(eval(data[0]).keys())
  difference_keys = []
  Kinds_of_rows = []
  for entry in data:
    try:
      entry = eval(entry)
      entry_keys = set(entry.keys())
      updated_data.append(entry)
      if len(entry_keys.difference(first_entry_data_keys)):
        Kinds_of_rows.append(entry)
        difference_keys.append(entry_keys.difference(first_entry_data_keys))
    except:
      continue
  return Kinds_of_rows, difference_keys, updated_data

In [0]:
def possbible_columns_in_df(data):
  """
  This function returns all possible columns to add in the dataframe. These column values are retrived by combining all the keys present in the json file.
  """
  Kinds_of_rows, difference_keys, updated_data = get_all_key_combinations_all_rows(data)
  extra_keys_in_some_entries = set().union(*difference_keys)
  common_columns_amongst_all_rows = eval(data[0]).keys()
  all_possible_columns = list(set(common_columns_amongst_all_rows).union(extra_keys_in_some_entries))
  return all_possible_columns, updated_data


all_possible_columns, updated_data = possbible_columns_in_df(data)
  
  

In [0]:
data_df = pd.DataFrame(columns = all_possible_columns)

In [0]:
def append_rows_to_df(data_df, data):
  
  """
  This function will append all the json entries as rows in the dataframe
  
  Parameters
  ----------
  data_df: Df to append the rows
  
  data: A list of dictionaries which are to be appended as rows
  
  Returns:
  data_df: Updated datafarme with rows appended
  
  """
  data_df = data_df.append(data, ignore_index=True)
  return data_df
  

In [0]:
updated_df = append_rows_to_df(data_df, updated_data)

In [103]:
updated_df.head()

,signature,fl,mltcount,start,sort,filter_service,excludesynonym,filter,resultcount,logtime,rows,referer,subuid,expressresume,highlight,queryany,searchwithin,queryexcludes,search_from,channel,_id,query,queryall
0,b8a69c82819dcb276999275e501470c1,"id,upt",NaN,0,upt,"{'cat': '1000,907,1,2,3,786,4,5,6,7,8,9,10,11,...",default,"{'refine_flag': '1', 'maxSal': 100, 'roles': '...",141,2019-05-10 00:00:11,160,SBR,643048,"{'sort': 'upt', 'seekerservices': 'ER1', 'rows...",true,NaN,NaN,NaN,Resume Search,1,5cd471b51ce48c8d454bb62c,NaN,NaN
1,967642e6878e9ca2cf922c8777d37226,"id,upt",NaN,0,rel,"{'cat': '1000,907,1,2,3,786,4,5,6,7,8,9,10,11,...",default,"{'maxSal': 150, 'primskill25': 'OR', 'locCondi...",35,2019-05-10 00:00:18,40,PWR,642343,"{'sort': 'rel', 'seekerservices': 'ER1', 'rows...",true,NaN,contents,NaN,Resume Search,1,5cd471bbd8228c0a15122ddf,NaN,NaN
2,d8a5dfd7f35a420ca26da2cb3ec40eb1,"id,upt",NaN,0,rel,"{'cat': '1000,907,1,2,3,786,4,5,6,7,8,9,10,11,...",default,"{'refine_flag': '1', 'maxSal': 150, 'primskill...",661,2019-05-10 00:00:41,40,PWR,642343,"{'sort': 'rel', 'seekerservices': 'ER1', 'rows...",true,NaN,contents,NaN,Resume Search,1,5cd471d357ad8c0d292cba2e,NaN,NaN
3,7efe34da00a06886d25acd38cdea07e3,"id,upt",NaN,0,rel,"{'cl': '167,171,249,183,190,254', 'pref': '167...",default,"{'primskill25': 'OR', 'locCondition': 'OR', 'n...",155,2019-05-10 00:00:44,40,PWR,601634,"{'sort': 'rel', 'seekerservices': 'ER6', 'rows...",true,Software Development Manager OR Technical Del...,contents,NaN,Resume Search,4,5cd471d81ce48c8d454bb62d,NaN,NaN
4,18249bb7ff94e5c9d666833ae0380a33,"id,upt",NaN,0,rel,NaN,default,"{'refine_flag': '1', 'primskill25': 'OR', 'ref...",413,2019-05-10 00:00:48,40,PWR,492803,"{'sort': 'rel', 'seekerservices': 'ER1', 'rows...",true,"""HR Business Partner"",",contents,NaN,Resume Search,1,5cd471d96e4d8cf964c152e6,NaN,NaN


In [0]:
def replace_multiple(main_string, to_be_replaces, new_sring):
  """
  This function replaces multiple substrings in a text by a replacement string
  
  Parameters
  ----------
  main_string: Text string
  
  to_be_replaces: List of substrings to be replaced in the text, for example: [" or ", " and "]
  
  new_sring: 
  
  """
  # Iterate over the strings to be replaced
  for elem in to_be_replaces :
    # Check if string is in the main string
    if elem in main_string :
      # Replace the string
      main_string = main_string.replace(elem, new_sring)
    
  return main_string

In [0]:
def preprocess_column_text(input_df, column_name):
  
  """
  This function pre-processes the given column by vectorized operations:
  
  1. Replace all the characters except alphabets and comma by white space
  2. Convert the whole string to lowercase characters
  3. Replace ' or ' and ' and ' sub strings in the text by ' , '
  4. Split the string by comma character
  5. After step 4, you get a list of words for each text, now strip all the whitespaces in each word at start and end
  
  Parameters
  ----------
  df: Dataframe object whose column is to be pre-processed
  column_name: Name of the column which is to be pre-processed in the df
  
  Returns:
  df: Updated df with pre-processed column
  
  """
  df = input_df.copy()
  df[column_name] = df[column_name].apply(lambda x: re.sub('[^,a-zA-Z]', " ", x) if isinstance(x, str) else "Not String")
  df[column_name] = df[column_name].apply(lambda x: x.lower())
  df[column_name] = df[column_name].apply(lambda x: replace_multiple(x, [' or ', ' and '], " , "))
  df[column_name] = df[column_name].apply(lambda x: x.split(','))
  df[column_name] = df[column_name].apply(lambda x: [y.strip() for y in x])
  return df

In [0]:
pre_processed_df = preprocess_column_text(updated_df, 'queryany')

In [108]:
pre_processed_df['queryany'].head(10)

0                                         [not string]
1                                         [not string]
2                                         [not string]
3    [software development manager, technical deliv...
4                              [hr business partner, ]
5                                      [it software, ]
6    [durability analysis, structural analysis, aba...
7                                         [not string]
8    [durability analysis, structural analysis, aba...
9                [active directory, group policy, dns]
Name: queryany, dtype: object